In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import torch

from itertools import chain

import matplotlib.pyplot as plt
%matplotlib inline

In [174]:
import sys

sys.path.append('..')

import utils

from importlib import reload

reload(utils)

from utils import dataset, applier, domain, pipe

reload(dataset)
reload(applier)
reload(domain)
reload(pipe)

from utils.dataset import LabeledDataset

In [4]:
data_path = "/Users/sergmiller/Documents/my/IDAO-2021/data/idao_dataset/"

In [47]:
train_path = os.path.join(data_path, "train")

In [151]:
sample_dataset = dataset.build_dataset(train_path, limit=10)

100%|██████████| 10/10 [00:00<00:00, 133.52it/s]


In [10]:
sample_dataset.save('test.npz')
sample_dataset = dataset.LabeledDataset.load('test.npz')

In [152]:
sample_dataset.samples.shape

(10, 576, 576)

In [12]:
import torchvision.models as models
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
model = models.mobilenet_v3_small(pretrained=True)

In [13]:
torch.set_num_threads(1)

In [14]:
torch.get_num_threads()

1

In [154]:
train_emb_dataset = LabeledDataset.load('../data/mobilenet_v3_small_embeddings.npz')


In [155]:
train_emb_dataset.samples.shape

(13404, 1000)

In [80]:
def apply_random_projection(data : np.array, to_dim : int = 10, seed : int = 0):
    assert 2 == len(data.shape)
    np.random.seed(seed)
    N = data.shape[1]
    proj_matrix = np.random.randn(N, to_dim) / np.sqrt(N)
    print(np.mean(proj_matrix), np.std(proj_matrix))
    data_proj = np.matmul(data, proj_matrix)
    return data_proj

In [156]:
proj_emb_dataset = copy.copy(train_emb_dataset)
proj_emb_dataset.samples = apply_random_projection(
    train_emb_dataset.samples, to_dim=50)

-0.00011997710679547996 0.03151695476906101


In [139]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score

In [163]:
params = {
    "penalty": 'l1',
    "solver": 'saga',
    "verbose": 10,
    "max_iter": 100,
    "tol": 1e-5,
    "scoring" : 'roc_auc',
    "random_state": 0,
    "cv": 10,
    "n_jobs": 7,
    "refit": False
}

In [149]:
ids = np.arange(len(train_proj_emb_dataset.samples))
np.random.seed(0)
np.random.shuffle(ids)
N = int(len(ids) * 0.9)
train_ids = ids[:N]
val_ids = ids[N:]
print(train_ids[:10], val_ids[:10])

[10869  3245 13299 10980  1103  5091 10134  6859  4516  4422] [ 1057 10013  6979  7505    60  6173  4878    35 11686  7315]


In [164]:
train_proj_emb_dataset = proj_emb_dataset.subset(train_ids)
val_proj_emb_dataset = proj_emb_dataset.subset(val_ids)

In [165]:
cv_label1 = LogisticRegressionCV(**params).fit(
    X=proj_emb_dataset.samples,
    y=(proj_emb_dataset.labels[:, 0] == 'NR').astype(int))

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


max_iter reached after 1 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=7)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=7)]: Done   1 out of   1 | elapsed:    0.8s finished


In [167]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(
    y_true=(val_proj_emb_dataset.labels[:, 0] == 'NR').astype(int),
    y_score=
)

In [127]:
np.mean(cv_label1.scores_[1], axis=0)  # new

array([0.5       , 0.5       , 0.76982608, 0.86866077, 0.9326519 ,
       0.94429291, 0.94827148, 0.95034892, 0.95169425, 0.9526548 ])

In [124]:
np.mean(cv_label1.scores_[1], axis=0)  # old

array([0.5       , 0.5       , 0.7952889 , 0.87402804, 0.94469283,
       0.95739159, 0.96157609, 0.96381906, 0.96529288, 0.96639186])

In [128]:
cv_label1.get_params()

{'Cs': 10,
 'class_weight': None,
 'cv': 10,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1.0,
 'l1_ratios': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': 7,
 'penalty': 'l1',
 'random_state': 0,
 'refit': False,
 'scoring': 'roc_auc',
 'solver': 'saga',
 'tol': 1e-05,
 'verbose': 10}

In [129]:
import copy

classes = [1, 10, 20, 3, 30, 6]

def mae_scorer(estimator, X, y):
    y_pred = np.array([classes[int(_y)] for _y in estimator.predict(X)])
    y = np.array([classes[_y] for _y in y])
    return -np.mean(np.abs(y_pred - y))

params2 = copy.copy(params)

params2['scoring'] = mae_scorer
# params2['class_weight'] = {str(c) : c for c in classes}
cv_label2 = LogisticRegressionCV(**params2).fit(
    train_proj_emb_dataset.samples,
    train_proj_emb_dataset.labels[:, 1])

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds


convergence after 4 epochs took 0 seconds
convergence after 4 epochs took 0 seconds
convergence after 4 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 5 epochs took 0 seconds
convergence after 5 epochs took 0 seconds
convergence after 5 epochs took 0 seconds
convergence after 5 epochs took 0 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds
max_iter reached after 4 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

...

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 3 seconds
max_iter reached after 3 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 3 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:  1.1min finished


In [24]:
np.mean(cv_label2.scores_['1'],axis=0) # mini weighted

array([7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.7, 5.7, 5.7, 5.7])

In [130]:
np.mean(cv_label2.scores_['1'],axis=0) # not weighted proj

array([-9.31475597, -8.29894242, -2.84744388, -1.16951128, -0.74142058,
       -0.62891488, -0.60055984, -0.58071633, -0.56407921, -0.55960338])

In [142]:
np.mean(cv_label2.scores_['1'],axis=0) # not weighted

array([10.68017027, 10.68017027, 10.09729052,  9.80483741,  9.63690119,
        9.57527796,  9.54491318,  9.53208126,  9.52514296,  9.51149031])

In [144]:
np.mean(cv_label2.scores_['1'],axis=0) # weighted

array([ 9.59965186,  9.4570564 ,  9.96051226, 10.08321655,  9.88160343,
        9.84861552,  9.80512292,  9.85751204,  9.71321238,  9.77365697])

In [106]:
pub_test_dataset = dataset.build_dataset(
    os.path.join(data_path, "public_test"),
    sample_processor=domain.process_test_sample)

100%|██████████| 1502/1502 [00:11<00:00, 133.88it/s]


In [107]:
private_test_dataset = dataset.build_dataset(
    os.path.join(data_path, "private_test"),
    sample_processor=domain.process_test_sample)

100%|██████████| 15062/15062 [01:59<00:00, 125.82it/s]


In [175]:
class ModelWithProjection:
    def __init__(self, model):
        self.model = model
    def predict(self, x):
        x_proj = apply_random_projection(x, to_dim=50)
        return self.model.predict(x_proj)
    def predict_proba(self, x):
        x_proj = apply_random_projection(x, to_dim=50)
        return self.model.predict_proba(x_proj)

In [176]:
model1 = ModelWithProjection(cv_label1)
model2 = ModelWithProjection(cv_label2)

In [177]:
sample_res = pipe.apply_all_models_to_test_dataset(
    sample_dataset,
    model1,
    model2,
    'sample')

100%|██████████| 1/1 [00:00<00:00, 3584.88it/s]


-0.00011997710679547996 0.03151695476906101
-0.00011997710679547996 0.03151695476906101


In [178]:
sample_res.tags.shape, sample_res.samples.shape

((10,), (10, 1000))

In [179]:
pub_test_predictions = pipe.apply_all_models_to_test_dataset(
    pub_test_dataset, model1, model2, 'pub_test')

100%|██████████| 94/94 [00:29<00:00,  3.17it/s]


-0.00011997710679547996 0.03151695476906101
-0.00011997710679547996 0.03151695476906101


In [180]:
private_test_predictions = pipe.apply_all_models_to_test_dataset(
    private_test_dataset, model1, model2, 'private_test')

100%|██████████| 942/942 [06:22<00:00,  2.46it/s]


-0.00011997710679547996 0.03151695476906101
-0.00011997710679547996 0.03151695476906101


In [191]:
train_emb_dataset.labels[0]

array(['NR', '1'], dtype='<U21')

In [190]:
model1.predict_proba(train_emb_dataset.samples[0].reshape(1, -1))

-0.00011997710679547996 0.03151695476906101


array([[0.17074507, 0.82925493]])

In [196]:
model2.predict(train_emb_dataset.samples[0].reshape(1, -1))

-0.00011997710679547996 0.03151695476906101


array(['1'], dtype='<U21')

In [183]:
submit = dataset.LabeledDataset.merge(
    pub_test_predictions, private_test_predictions)

In [198]:
submit.labels[:, 0] = np.array([1 - float(x) for x in submit.labels[:, 0]])

In [199]:
submit.labels[:10]

array([['0.1714462987687032', '6'],
       ['0.9999890978353596', '30'],
       ['0.8916120295757206', '30'],
       ['0.9999999749217294', '30'],
       ['0.1487708185318909', '6'],
       ['0.10958024790485266', '20'],
       ['0.9999672889721145', '30'],
       ['0.0025010287196323144', '20'],
       ['0.7359871100580124', '3'],
       ['0.05572645307588686', '1']], dtype='<U32')

In [200]:
dataset.dataset2submit_csv(submit,
    "../submits/1:log_reg_2:log_reg_multiclass_not_weighted_over_proj50_refit_false_seed_0_mobilenet_v3_small_embds.csv")

In [192]:
submit.tags.shape

(16564,)

In [193]:
submit.samples.shape

(16564, 1000)

In [194]:
pub_test_predictions.tags.shape, pub_test_predictions.samples.shape

((1502,), (1502, 1000))

In [195]:
private_test_predictions.tags.shape, private_test_predictions.samples.shape

((15062,), (15062, 1000))

In [244]:
model = applier.mobilenet_v3_small
model.train(False)

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_r

In [245]:
torch.save(model.state_dict(), "mobilenet_state_dict")

In [246]:
import torchvision
model2 = mobilenet_v3_small()

In [247]:
model2.load_state_dict(torch.load("mobilenet_state_dict"))

<All keys matched successfully>

In [248]:
np.random.seed(0)
t = np.random.random((1,3,500,500))
model2.train(False)
print(np.mean(t), np.std(t))
with torch.no_grad():
    res = model2(torch.Tensor(t)).detach().numpy()
print(np.mean(res), np.std(res))

0.5002803215266387 0.2887019402063319
0.46069193 1.7000295


In [249]:
np.random.seed(0)
t = np.random.random((1,3,500,500))
model.train(False)
print(np.mean(t), np.std(t))
with torch.no_grad():
    res = model(torch.Tensor(t)).detach().numpy()
print(np.mean(res), np.std(res))

0.5002803215266387 0.2887019402063319
0.46069193 1.7000295


In [214]:
model

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_r

In [231]:
import torchvision
mobilenet = torchvision.models.mobilenet_v3_small()
mobilenet.load_state_dict(torch.load("mobilenet_state_dict"))

np.random.seed(0)
t = np.random.random((1,3,500,500))
model2.train(False)
print(np.mean(t), np.std(t))
with torch.no_grad():
    res = mobilenet(torch.Tensor(t)).detach().numpy()
print(np.mean(res), np.std(res))



import time
start = time.time()
for i in np.arange(10):
    t = np.random.random((32,3,500,500))
    with torch.no_grad():
        res = mobilenet(torch.Tensor(t)).detach().numpy()
    
print('OK, run {}s'.format(time.time() - start))

0.5002803215266387 0.2887019402063319
3.4595482 0.81053764
OK, run 25.1478328704834s


In [242]:
import sys

sys.path.append('..')
from submition_track2.test_in_submit.mobilenet_v3 import mobilenet_v3_small

In [243]:
mobilenet = mobilenet_v3_small()